In [28]:
import numpy as np
import math
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
# M1/m2 Mac上での警告回避
from tensorflow.compat.v1.keras.optimizers import Adam as LegacyAdam

tf.config.run_functions_eagerly(True) # eager executionを有効に
tf.data.experimental.enable_debug_mode() # なんか警告でたのでデバッグモードを有効に


In [29]:
# params

# energy
x_max = 10 # xの最大値
x_list = np.linspace(0, x_max, 11) # xの値域
m2 = 1 # EFTのcutoff scale
m2_list = m2 * (1 + x_list) 
# spin
J_max = 40 # Jの最大値
J_list = np.arange(0, J_max, 2) # Jの値域
# spacetime dimension
d = 10

In [30]:
# g_k関数, null constraint
def g2(m2, J):
    return 1 / (m2 ** 2)

def mathcalJ2(J):
    return J * (J + d - 3)

def g3(m2, J):
    return ( 3 - (4/(d-2)) * mathcalJ2(J) ) / ( m2 ** 3 )

def n4(m2, J):
    return ( mathcalJ2(J) * ( 2*mathcalJ2(J)- 5*d + 4 ) ) / ( m2 ** 4 )

In [31]:
# normalization constant
def nJd(J):
    return ( (4*np.pi)**(d/2) * (d+2*J-3) * (np.vectorize(math.gamma)(d+J-3)) ) / ( np.pi * np.vectorize(math.gamma)((d-2)/2)*np.vectorize(math.gamma)(J+1) )

In [32]:
# DataFrameを作成
df = pd.DataFrame(list(product(m2_list, J_list)), columns=['m2', 'J'])

# 各関数を DataFrame に格納
df['g2'] = g2(df['m2'], df['J'])
df['g3'] = g3(df['m2'], df['J'])
df['n4'] = n4(df['m2'], df['J'])
# df['nJd'] = nJd(d, df['J'])

In [33]:
df

,m2,J,g2,g3,n4
0,1.0,0,1.000000,3.000000,0.000000
1,1.0,2,1.000000,-6.000000,-180.000000
2,1.0,4,1.000000,-19.000000,1848.000000
3,1.0,6,1.000000,-36.000000,8580.000000
4,1.0,8,1.000000,-57.000000,23280.000000
...,...,...,...,...,...
215,11.0,30,0.008264,-0.414726,164.820709
216,11.0,32,0.008264,-0.466566,208.838194
217,11.0,34,0.008264,-0.521412,261.071512
218,11.0,36,0.008264,-0.579264,322.477973


In [34]:
# for test 
df['rho'] = 1

In [35]:
# 'J'列でグループ化して各Jの値ごとにDataFrameをまとめる
grouped_dfs = [grouped_df for _, grouped_df in df.groupby('J')]

In [36]:
# grouped_dfs[0]

In [37]:
def integrand(func):
    output_list = []
    for grouped_df in grouped_dfs:
        grouped_df['m2^((2-d)/2)'] = grouped_df['m2'] ** ((2-d)/2)
        output_list.append( grouped_df['m2^((2-d)/2)'] * grouped_df['rho'] * grouped_df[func] )
    return output_list

def summand(func):
    
    # integrandを、m2について台形近似で数値積分
    output_list = []
    for grouped_df in grouped_dfs:
        output_list.append( np.trapz( integrand(func=func), m2_list) )
    return output_list

In [38]:
# heavy average
def heavy_average(func):
    output_list = []
    for i in range(len(J_list)):
        output_list.append( nJd(J_list[i]) * summand(func=func)[i] )
    return np.sum(output_list)

In [39]:
# model
n_node = 4
model = Sequential([
    Dense(n_node, activation='relu', input_dim=2),
    Dense(1, activation='relu')
])


In [40]:
# input data
X_input = df[['m2', 'J']].values
y_input = tf.zeros(len(df))

In [41]:
# loss function
def custom_loss(y_true, y_pred):
    a2 = 1
    a3 = 1
    w4 = 10
    df['rho'] = model(X_input)
    return a2 * heavy_average(func='g2') + a3 * heavy_average(func='g3') + w4 * tf.square( heavy_average(func='n4') )


In [42]:
model.compile(optimizer=LegacyAdam(), loss=custom_loss)

In [43]:
# 学習
n_epochs = 300
for i in range(1, n_epochs+1): # 反復回数
    model.fit(X_input, y_input, epochs=n_epochs, verbose=0)

ValueError: No gradients provided for any variable: (['dense_2/kernel:0', 'dense_2/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense_2/kernel:0' shape=(2, 4) dtype=float32, numpy=
array([[ 0.42598104, -0.35039425,  0.8770454 ,  0.61137104],
       [-0.21979833, -0.60926485,  0.4290464 , -0.4527812 ]],
      dtype=float32)>), (None, <tf.Variable 'dense_2/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>), (None, <tf.Variable 'dense_3/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.88030344],
       [-0.2729767 ],
       [ 0.8087487 ],
       [ 0.776279  ]], dtype=float32)>), (None, <tf.Variable 'dense_3/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>)).